# Apply PSO on Matching Network Optimization
From separate code on PSO and Matching Network, it's time to merge these 2 to make an optimization for MN

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skrf as rf
from bitarray import bitarray

----------------------------------------------------------
## I. Define Matching Network

Still, suppose I have a MN of 5 component like this:

*-------------------------Z3---------------Z1---------------*

     |                  |             |
           
    Z_out               Z2             Z_load
           
     |                  |             |
      
*---------------------------------------------------------------*

and Z_load is not matched at 50Ohm, so with help from MN, I expect to get Z_out matched

Let's try to simplify the parallel or series type of a circuit

In [ ]:
def parallel(Z1, Z2):
    return 1/(1/Z1 + 1/Z2)

def series(Z1, Z2): 
    return Z1 + Z2

Then, I assign type (R/L/C), connection orientation (ser/par), lowest and highest values of 5 components, respectively:

In [ ]:
num_component = 3
component_type = np.zeros(num_component)
component_topology = np.zeros(num_component)
range_C = (1e-12, 100e-12)
range_distance_C = range_C[1] - range_C[0]
range_L = (1e-9, 100e-9)
range_distance_L = range_L[1] - range_L[0]
component_topology

And the function to calculate output impedance from Z load and MN

*note: for component type: 0 - C, 1 - L, 2 - R; for component placement: 0 - series, 1 - parallel*

In [ ]:
def out_impedance(num_component, components, freq, Z_load):
    Z = np.zeros((num_component+1,),dtype = complex)
    Z_out = np.zeros((num_component+1,),dtype = complex)
    # calculate impedance of each component
    Z[0] = Z_load
    #print (Z[0],end = '   ')
    for i in range(num_component):
        if components[i][0] == 0: # C
            Z[i+1] = 1/(1j*2*np.pi*freq*components[i][2])
        elif components[i][0] == 1: # L
            Z[i+1] = 1j*2*np.pi*freq*components[i][2]
        else: # R
            Z[i+1] = components[i][2]
    # calculate result
    Z_out[0] = Z_load
    for i in range(num_component):
        if components[i][1] == 0: # series
            Z_out[i+1] = series(Z_out[i], Z[i+1])
        else:                     # parallel
            Z_out[i+1] = parallel(Z_out[i], Z[i+1])
        #print ('\n Zout', i+1,' :', Z_out[i+1])
    return Z_out[num_component]

---------------------------------------------
## II. Apply PSO
General parameters of PSO are defined here, meaning of them is *PSOsinc* notebook


In [ ]:
w = 0.3
c1 = 2
c2 = 2
max_iter = 20
#freq = 1e+9
#Z_load = 20
Z0 = 50
ntwk = rf.Network('dualpolar_0409_HFSSDesign1.s1p')
ntwk.plot_s_smith()

### First step
Define the Solution Space

10 bees, or better-10 particles, will be assigned to the task to look for the best position in their so-called Solution Space. This space is 5D, as each agent needs 5 variables to define its location, seem unrealistic hah? :v

In [ ]:
agent_quantity = 10
space_dimention = num_component
agent_bound = np.zeros([num_component,2])
def agent_bound_maker(component_type):
    for i in range(space_dimention):
        if component_type[i] == 0: # Comp C
            agent_bound[i] = range_C
        else:                      # Comp L
            agent_bound[i] = range_L
    return agent_bound
    
agent_bound_maker([0,1,0])

### Second step
Define a fitness function. In this case, it's the S11 at given frequency

In [ ]:
def fitness(agent, component_type, component_topology):
    fitness = 0
    components = np.zeros([num_component,3])
    for i in range(num_component):
        # Component type
        components[i][0] = component_type[i]
        # Component placement
        components[i][1] = component_topology[i]
        # Component value
        components[i][2] = agent[i]
        
    for i in range(480,561,1):
        Z_out = out_impedance(num_component, components, 1e9 + i*5e6, ntwk.z[i][0][0])
        fitness += np.abs((Z_out - Z0)/(Z_out + Z0))
    return 1 - fitness/81
# Check fitness function
fitness([10e-12, 20e-12, 30e-12],[0,1,0],[0,1,0])

### Third step 
Initialize Random Swam Location and Velocities

In [ ]:
agent_location = np.zeros([agent_quantity, space_dimention])

agent_fitness = np.zeros([agent_quantity,1])
pbest = np.zeros([agent_quantity,1])

def agent_location_maker(component_type):
    for i in range(space_dimention):
        if component_type[i] == 0:
            for j in range(agent_quantity):
                agent_location[j][i] = range_distance_C*np.random.rand()+range_C[0]
        elif component_type[i] == 1:
            for j in range(agent_quantity):
                agent_location[j][i] = range_distance_L*np.random.rand()+range_L[0]
    return agent_location
#agent_location_maker([0,1,0,1,0])

In [ ]:
agent_velocity = np.zeros([agent_quantity, space_dimention])
def agent_velocity_maker(component_type):
    for i in range(space_dimention):
        if component_type[i] == 0:
            for j in range(agent_quantity):
                agent_velocity[j][i] = range_distance_C*np.random.rand()+range_C[0]
        elif component_type[i] == 1:
            for j in range(agent_quantity):
                agent_velocity[j][i] = range_distance_L*np.random.rand()+range_L[0]
    return agent_velocity
#agent_velocity_maker([0,1,0,1,0])

### Fourth step and Fifth step will be looped over topologies and types

In [ ]:
for khai in range(np.power(2,2*num_component)):
    #Define component_type and component_topology
    for i in range(num_component):
        component_topology[i] = bitarray(np.binary_repr(khai,2*num_component))[i]
        component_type[i] = bitarray(np.binary_repr(khai,2*num_component))[i+3]
    print('\n',khai,'. Loop')
    
    #Define boundary
    agent_bound = agent_bound_maker(component_type)
    
    #Generate random Swam locations
    agent_location = agent_location_maker(component_type)
    #pbest at first is also the first swam
    agent_pbest = agent_location
    #Generate velocities
    agent_velocity = agent_velocity_maker(component_type)
    
    # Evaluate the Particles's Fitness, Compare to gbest, pbest
    for i in range(0,agent_quantity):
        agent_fitness[i] = fitness(agent_location[i],component_type, component_topology)
        if agent_fitness[i] > pbest[i]:
            pbest[i] = agent_fitness[i]
            agent_pbest[i] = agent_location[i]
            
    gbest = np.zeros(1)
    gbest[0] = pbest.max()
    gbest_position = pbest.argmax()
    print('\n Global best', gbest[0])
    print('\n Global best position', gbest_position)
    print('\n Agent at best position', agent_pbest[gbest_position])
    
    # Update the Particle's velocity
    for i in range(agent_quantity):
        agent_velocity[i] = w*agent_velocity[i] + c1*np.random.rand()*(agent_pbest[i] - agent_location[i]) + c2*np.random.rand()*(agent_pbest[gbest_position]- agent_location[i])
    
    #Move the particle
    agent_location = agent_location + agent_velocity
    for i in range(agent_quantity):
        for j in range(space_dimention):
            if agent_location[i][j] > agent_bound[j][1]:
                agent_location[i][j] = 2*agent_bound[j][1] - agent_location[i][j]
            elif agent_location[i][j] < agent_bound[j][0]:
                agent_location[i][j] = 2*agent_bound[j][0] - agent_location[i][j]
    
    #Fifth step
    converge_flag = 0
    for k in range(max_iter):
        if gbest[k] > 0.75:
            converge_flag = 1
            print('\n Converged!!! Hura!!')
            break
        # Step 4a
        for i in range(agent_quantity):
            agent_fitness[i] = fitness(agent_location[i],component_type, component_topology)
            if agent_fitness[i] > pbest[i]:
                pbest[i] = agent_fitness[i]
                agent_pbest[i] = agent_location[i]
    
        gbest = np.append(gbest, pbest.max())
        gbest_position = pbest.argmax()
        # Step 4b
        for i in range(agent_quantity):
            agent_velocity[i] = w*agent_velocity[i] + c1*np.random.rand()*(agent_pbest[i] - agent_location[i]) + c2*np.random.rand()*(agent_pbest[gbest_position]- agent_location[i])
    
        # Step 4c
        agent_location = agent_location + agent_velocity
        for i in range(agent_quantity):
            for j in range(space_dimention):
                if agent_location[i][j] > agent_bound[j][1]:
                    agent_location[i][j] = 2*agent_bound[j][1] - agent_location[i][j]
                elif agent_location[i][j] < agent_bound[j][0]:
                    agent_location[i][j] = 2*agent_bound[j][0] - agent_location[i][j]
        #print('Iteration: ', k)
        #print('Gbest in this iteration: ', gbest)

    if converge_flag == 0:
        k +=1 
    
    iterate = np.arange(k+1)
    plt.figure(khai)
    plt.plot(iterate,gbest)
    plt.xlabel('iterations(n)')
    plt.ylabel('gbest')
    plt.show()
    print('\n Agent at best position', agent_pbest[gbest_position])            
    print('\n Gbest value is ', gbest[k])

-----------------------------------------------------------
# III. Result
Final result is shown here, good or not, it've already taken a lot of time. Let's try to draw a graph to see the convergence progress